In [1]:
## Module imports 

import numpy as np

import matplotlib.pyplot as plt
plt.rcParams["text.usetex"] = False
from matplotlib import cm, colors
from matplotlib.patches import Ellipse
import matplotlib as mpl

from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
from astropy.visualization.wcsaxes import SphericalCircle

from scipy.ndimage import gaussian_filter1d

import os

import requests

from desispec.io import read_spectra
from desispec.spectra import stack as specstack
from desispec.coaddition import coadd_cameras

In [2]:
## Define constants

h = 1
H0 = 100*h
c = 3e5
q0 = 0.2

# Visual Inspection
Trying to do the things to make Prospect work.

In [3]:
tfuji_all = Table.read("/global/cfs/projectdirs/desi/science/td/pv/desi_pv_tf_fuji_healpix.fits")
tfuji = tfuji_all[tfuji_all["TARGETID"] > 0]
tfuji[:3]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC
int64,float64,float64,int64,bytes3,float64,float64,int64,float64,bytes63,bytes3,int64,float64,float64
1079550234591232,194.390863195343,27.5157211790145,10378,sv3,1.1235686466514,7.31685779475115e-05,4,3.28414569795132,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,EXT,662902,194.39086319534337,27.51572117901454
1092744374124544,194.390863195343,27.5157211790145,10378,sv3,0.686773088332363,6.9756676262104e-05,4,0.786607094109058,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.39086319534337,27.51572117901454
1092744374124546,194.364461113654,27.5037185881314,10378,sv3,0.0242933923052181,4.95233472646785e-05,0,95.428411073226,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.36446111365385,27.50371858813136


In [4]:
## Lewis's code to make a txt file of fuji data.

def create_list_targetids(filename, data, filetype, separate=False):
    """
    Creates a fits or txt files of the targetid names. 
    If separate=True, the targetids will be separated by " separate_by " 
    At least one is required to run prospect. 
    
    Parameters
    ----------
    data : astropy.Table
    filename: The beggining of the exported new filename
    filetype: Do you want a fits file or txt file?
    separate: if True, can create several files separated by a column
    
    Returns
    -------
    Creates a fits file or txt file with list of targetids.
    """
    if separate == True:
        colname = str(input("What would you like to separate by?")).upper()
        
        for separate_by in np.unique(data[f'{colname}']):

            PV_data = data[data[f'{colname}']==separate_by]
            TARGETIDS = []
            for target_id in np.unique(PV_data['TARGETID']):
                TARGETIDS.append(target_id)

            # Ensuring the data is the correct type  
            TARGETIDS = np.array(TARGETIDS, dtype = np.int64)

            if filetype == 'fits':
                # Creating the fits file. Two different ways of doing it
                """hdu = fits.PrimaryHDU(TARGETIDS)
                hdu.writeto(f'{filename}_{healpix}.fits', overwrite=True)"""
                t = Table([TARGETIDS], names=('TARGETID',), 
                          meta={'name': f'TARGETID table TF FUJI {colname} separated. {colname} {separate_by}'})
                t.write(f'{filename}_{colname}_{separate_by}.fits', format = 'fits', overwrite=True)
            else:
                # Creating txt file
                np.savetxt(f'{filename}_{colname}_{separate_by}.txt', TARGETIDS, delimiter = ',', fmt = '%d')
    
    else:
        TARGETIDS = []
        for target_id in np.unique(data['TARGETID']):
            TARGETIDS.append(target_id)

        # Ensuring the data is the correct type  
        TARGETIDS = np.array(TARGETIDS, dtype = np.int64)

        if filetype == 'fits':
            # Creating the fits file. Two different ways of doing it
            """hdu = fits.PrimaryHDU(TARGETIDS)
            hdu.writeto(f'{filename}.fits', overwrite=True)"""
            t = Table([TARGETIDS], names=('TARGETID',), meta={'name': f'TARGETID table after quality cuts TF FUJI HEALPIX'})
            t.write(f'{filename}.fits', format = 'fits', overwrite=True)
        else:
            # Creating txt file
            np.savetxt(f'{filename}.txt', TARGETIDS, delimiter = ',', fmt = '%d')
    
    # No return


In [10]:
create_list_targetids("tfuji_IDs", tfuji, "txt", separate=False)